## Using a User-owned S3 Bucket to Stage Results

In [ ]:
import boto3
import datetime as dt
import helper
import json
import rasterio
import rasterio.plot
import netCDF4 as nc4
from matplotlib import pyplot as plt
import numpy as np
import re
import requests
import sys
sys.path.append('..')
!{sys.executable} -m pip install -q -r ../requirements/examples.txt

# Install harmony-py requirements.  Not necessary if you ran `pip install harmony-py` in your kernel  
!{sys.executable} -m pip install -q -r ../requirements/core.txt

from harmony import BBox, Client, Collection, LinkType, Request, s3_components, Environment

In [ ]:
# the bucket we will use to store our results
staging_bucket = 'jn-staging-test4'

# create an AWS session using our profile
session = boto3.Session(profile_name='harmony-sit')

# check to see if the bucket already exists - create it if it does not
s3_client = session.client('s3')
try:
    response = s3_client.head_bucket(
        Bucket=staging_bucket
    )
except:
    print(f'Creating bucket {staging_bucket}')
    response = s3_client.create_bucket(
        Bucket=staging_bucket, 
        CreateBucketConfiguration={
            'LocationConstraint': 'us-west-2'
        },
        ObjectOwnership='BucketOwnerEnforced')
    response

In [ ]:
request = Request(
    collection=Collection(id='C1940472420-POCLOUD'),
    spatial=BBox(-160, -80, 160, 80),
    variables=['bathymetry'],
    max_results=3,
    concatenate=False,
    destination_url=f's3://{staging_bucket}/foo'
)

request.is_valid()

In [ ]:
# create a Harmomy client with which to make requests
harmony_client = Client(env=Environment.PROD)# assumes .netrc usage

In [ ]:
job_id = None
bucket_policy_url = None
try:
    job_id = harmony_client.submit(request)
    print(job_id)
except Exception as e:
    error = e.args[1]
    bucket_policy_url = re.search("https:\/\/.*", error).group()
    print(error)

In [ ]:
if bucket_policy_url:
    # get the bucket policy from harmony
    policy = requests.get(bucket_policy_url).json()
    # set the policy on the bucket
    response = s3_client.put_bucket_policy(
        Bucket=staging_bucket,
        Policy=json.dumps(policy)
    )
    print(response)

Now let's get the data URLs from Harmony, but request S3 URLs instead of the default HTTPS.

In [ ]:
results = harmony_client.result_urls(job_id, link_type=LinkType.s3)

Now we'll use the AWS Python library boto3 to download our results from the S3 bucket

In [ ]:
for url in results:
    print(url)
    bucket, obj, fn = s3_components(url)
    with open(fn, 'wb') as f:
        s3_client.download_fileobj(bucket, obj, f)
        helper.show_result(f.name)
        # nc4_file=nc4.Dataset(f.name)
        # arrays = []
        # for var in ['bathymetry']:
        #     ds = nc4_file.variables[var][0]
        #     arrays.append(ds)
        # plt.imshow(np.dstack(arrays))